In [4]:
import flask
import io
import string
import time
import os
import numpy as np
import tensorflow as tf
import pickle, joblib
from PIL import Image
from flask import Flask, jsonify, request
from prep_image_module import prep_img
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
VGG_model  = tf.keras.models.load_model('VGG_model.h5')
model = joblib.load('XGBOOST.pkl')

# def prepare_image(img):
#     img = Image.open(io.BytesIO(img))
#     img = img.resize((224, 224))
#     img = np.array(img)
#     img = np.expand_dims(img, 0)
#     return img


def predict(path):
    # img1 = cv2.imread(path, cv2.IMREAD_COLOR)
    img1 = prep_img(path)
    img1 = cv2.resize(img1, (SIZE, SIZE))
    img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2BGR)
    
    img1 = img1/255
    plt.imshow(img1)
    input_img = np.expand_dims(img1, axis=0)
    input_img_feature=VGG_model.predict(input_img)
    input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
    prediction = model.predict(input_img_features)[0] 
    prediction = le.inverse_transform([prediction]) 
    return prediction

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def infer_image():
    if 'file' not in request.files:
        return "Please try again. The Image doesn't exist"
    
    file = request.files.get('file')

    if not file:
        return

    img_bytes = file.read()
    img = img_bytes

    return jsonify(prediction=predict(img))
    

@app.route('/', methods=['GET'])
def index():
    return 'Machine Learning Inference'


if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

In [5]:
! flask run

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Usage: flask run [OPTIONS]
Try 'flask run --help' for help.

Error: Could not locate a Flask application. You did not provide the "FLASK_APP" environment variable, and a "wsgi.py" or "app.py" module was not found in the current directory.
